In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Allen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Allen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
initialData = pd.read_csv('dbb4c4ff1f31-CAC+2022_Training+Data+Set+New.csv')

In [3]:
def removeOnes(df):
    rev = list(df['trans_desc'])
    for item in range(len(rev)):
        rev[item] = re.sub(" +", " ", rev[item])
        rev[item] = re.sub("111111XXXXXX1111 111111111111111", "", rev[item])
        rev[item] = re.sub("\?MCC=1111 11", "", rev[item])
        rev[item] = re.sub("111-111-1111", " ", rev[item])
        rev[item] = re.sub("111-1111111", "", rev[item])
        rev[item] = re.sub("11111", "", rev[item])
        rev[item] = re.sub("\?MCC=1111", "", rev[item])
        
        
    rev_location = list(df['default_location'])
    rev_location = [str(p) for p in rev_location]
    for item in range(len(rev_location)):
        rev_location[item] = re.sub("111-111-1111", "", rev_location[item])
        rev_location[item] = re.sub("111-1111111", "", rev_location[item])
        rev_location[item] = re.sub("11111", "", rev_location[item])
        
    
        
    for index, row in df.iterrows():
        df.at[index, 'trans_desc'] = rev[index]
        df.at[index, 'default_location'] = rev_location[index]
    return df

In [4]:
initialData = removeOnes(initialData)

In [5]:
def lowerCaps(df):
    df = df.applymap(lambda r: r.lower() if type(r) == str else r)
    return df

In [6]:
initialData = lowerCaps(initialData)

In [7]:
initialData['numCat'] = (initialData['Category'].apply(lambda c: 0 if c == 'comunication services' 
                                                       else 1 if c == 'education' 
                                                       else 2 if c == 'entertainment' 
                                                       else 3 if c == 'finance' 
                                                       else 4 if c == 'health and community services' 
                                                       else 5 if c == 'property and business services' 
                                                       else 6 if c == 'retail trade' 
                                                       else 7 if c == 'services to transport' 
                                                       else 9 if c == 'travel' 
                                                       else 8))

In [8]:
new = initialData.iloc[:, [0,2,4,7,8,12]]

In [9]:
initialData['congl2'] = initialData[new.columns[0:6]].apply(lambda x: ' | '.join(x.astype(str)), axis = 1)

In [10]:
initialData[0:1]

,sor,cdf_seq_no,trans_desc,merchant_cat_code,amt,db_cr_cd,payment_reporting_category,payment_category,is_international,default_brand,default_location,qrated_brand,coalesced_brand,Category,numCat,congl2
0,hh,t20110701260061756,recur debit crd pmt11/11 delta dental of a11 o...,6300.0,58.34,d,card,debit card,False,delta dental of a11 of,ar,delta dental,delta dental,finance,3,hh | recur debit crd pmt11/11 delta dental of ...


In [11]:
X = np.array(initialData.iloc[:, [15]])
y = np.array(initialData.iloc[:, [14]])

In [12]:
# Remove grammar, tokenize, and lemmatize the sentences
def clean_sentences(df):
    reviews = []

    for sent in (df['congl2']):
        
        # remove non-alphabetical characters
        text = re.sub('[^a-zA-Z]', ' ', sent)
        
        # tokenize sentence
        words = word_tokenize(text.lower())
        
        # remove stop words:
        new_words = [char for char in words if char.lower() not in stopwords.words('english')]
        
        # lemmatizing each word to its lemma
        lem = WordNetLemmatizer()
        lem_words = [lem.lemmatize(i) for i in new_words]
    
        reviews.append(lem_words)

    return(reviews)

In [13]:
train_sen = clean_sentences(initialData)

In [14]:
X_train,X_test,y_train,y_test = train_test_split(train_sen,y,test_size=0.2)

In [15]:
unique_words = set()
len_max = 0

for sent in (X_train):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)
        
print(len(list(unique_words)))
print(len_max)

29007
25


In [16]:
vocab_size = len(list(unique_words))
embedding_dim = 300
max_length = len_max
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [17]:
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [18]:
X_train = pad_sequences(X_train, maxlen=len_max)
X_test = pad_sequences(X_test, maxlen=len_max)

print(X_train.shape,X_test.shape)

(32000, 25) (8000, 25)


In [19]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=11)

In [20]:
y_train

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [21]:
y_train.shape

(32000, 11)

In [22]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes=11)

In [23]:
from keras.models import Sequential
from keras.layers import Dense,Bidirectional,LSTM,Activation,Dropout,Flatten, Conv1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, BatchNormalization, MaxPooling1D
from keras.regularizers import l2

In [27]:
model = Sequential([
    Embedding(29218, 300, input_length = 25),
    Conv1D(128, 2, padding='same',activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling1D(pool_size = 2),
    Dropout(0.2),
    Conv1D(64, 2, padding='same',activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling1D(pool_size = 2),
    Dropout(0.2),
    Flatten(),
    Dense(32, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(11,activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [25]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(min_delta = 0.001,
                               mode = 'max',
                               monitor = 'val_accuracy',
                               patience = 2)
callback = [early_stopping]

In [28]:
num_epochs = 30

history = model.fit(X_train,y_train,
                    validation_data = (X_test, y_test),
                    epochs = num_epochs,
                    batch_size = 256,
                    verbose = 1,
                    callbacks = callback)

Epoch 1/30
125/125 [==============================] - 10s 72ms/step - loss: 4.3839 - accuracy: 0.4797 - val_loss: 4.8011 - val_accuracy: 0.0081
Epoch 2/30
125/125 [==============================] - 9s 73ms/step - loss: 2.2548 - accuracy: 0.7937 - val_loss: 3.9087 - val_accuracy: 0.0322
Epoch 3/30
125/125 [==============================] - 10s 77ms/step - loss: 1.2182 - accuracy: 0.9047 - val_loss: 2.9614 - val_accuracy: 0.2961
Epoch 4/30
125/125 [==============================] - 12s 93ms/step - loss: 0.7383 - accuracy: 0.9399 - val_loss: 2.1847 - val_accuracy: 0.5565
Epoch 5/30
125/125 [==============================] - 12s 100ms/step - loss: 0.5058 - accuracy: 0.9546 - val_loss: 1.4687 - val_accuracy: 0.6619
Epoch 6/30
125/125 [==============================] - 13s 102ms/step - loss: 0.3751 - accuracy: 0.9631 - val_loss: 1.2530 - val_accuracy: 0.6964
Epoch 7/30
125/125 [==============================] - 13s 106ms/step - loss: 0.2988 - accuracy: 0.9687 - val_loss: 1.3936 - val_accurac